# Lab Workbook No 6: Geodemographics

## Challenge 1: Geodemographic Classification


The main goal of this geodemographic classification is to identify the vulnerable groups in Edinburgh for better service planning. 

In [ ]:
import geopandas as gpd

# Importing the shp file of the boundary of the 
# Scottish output area 2011 as a GeoDataFrame
gdf = gpd.read_file(
    "/UA/Lab 6 Data /output-area-2011-mhw/OutputArea2011_MHW.shp")

# Subsetting the boundary of the City of Edinburgh from the .shp file 
# (with the value of S12000036 in the council column)
EDI_ONLY_gdf = gdf[gdf["council"] == "S12000036"]
EDI_ONLY_gdf.to_file("EDI_ONLY.shp")

# Review the subset data frame
EDI_ONLY_gdf.head()

In [ ]:
import pandas as pd
import os

# importing the Scotland 2011 census data 
csv_directory = "/UA/Lab 6 Data /Census_raw_data"

# locate all .csv files and merge them into a single dataframe
csv_files = [file for file in os.listdir(csv_directory) 
             if file.endswith(".csv")]

merged_data = pd.DataFrame() # creating a empty dataframe

for csv_file in csv_files:
    csv_path = os.path.join(csv_directory, csv_file) 
    df_csv = pd.read_csv(csv_path, low_memory=False) 
    merged_data = pd.concat([merged_data, df_csv], axis=1)
    
# Saving the merged data into the following address
merged_data.to_csv("/UA/Lab 6 Data /merged_census_data.csv", 
                   index=False)

In [ ]:
# Merging the census data with the City of Edinburgh output area boundary

csv_path = "/UA/Lab 6 Data /merged_census_data.csv"
csv_data = pd.read_csv(csv_path, low_memory=False)

merged_data = EDI_ONLY_gdf.merge(csv_data, 
                                 left_on='code', 
                                 right_on='oa_code', 
                                 how='left')

I have selected the four topics of Health, Economic activity, Central heating and Household composition structure to better illustrate which area is more vulnerable and requires more resources of public services. I have selected four variables that are crucial for effectively segmenting neighbourhoods, such as the elderly who lives alone, People in poor health condition, residents with no central heating, and unemployed but economically active people. These variables could directly address specific dimensions in both social and economic vulnerability, highlighting areas where people are at higher risk of isolation, health deterioration, or poor living conditions.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Selecting four variables and plotting all four graphs into one figure
attributes_to_plot = ['One person household: Aged 65 and over',
                      'Bad health',
                      'No central heating',
                      'Economically active: Unemployed']

# Plotting a violin plot
plt.figure(figsize=(12, 8))

for i, attribute in enumerate(attributes_to_plot, 1):
    plt.subplot(2, 2, i)
    sns.violinplot(x=merged_data[attribute])
    plt.title(attribute)
    plt.xlabel('')
    plt.ylabel('')

plt.tight_layout()
plt.show()

In [ ]:
# Selecting four variables and plotting all four graphs into one figure
attributes_to_plot = ['One person household: Aged 65 and over',
                      'Bad health',
                      'No central heating',
                      'Economically active: Unemployed']

# Plotting a histogram
plt.figure(figsize=(12, 8))

for i, attribute in enumerate(attributes_to_plot, 1):
    plt.subplot(2, 2, i)
    sns.histplot(merged_data[attribute].astype(str), kde=True)
    plt.title(attribute)

plt.tight_layout()
plt.show()

Now we prepare and clean the data for standardisation and clustering.

In [ ]:
# Calculating the percentages of the selected variables

# Setting up function to transform the data into percentages
def calculate_percentages(dataframe, total_columns, value_columns):

    result_df = pd.DataFrame() # empty dataframe for the processed data 

    for total_col, value_col in zip(total_columns, value_columns):
        percentage_col_name = f"{value_col}_percentage"

        if total_col not in dataframe.columns:
            raise ValueError(
                f"Total column '{total_col}' not found in the DataFrame.")
            
        # Turning any empty values into numeric or NaN value
        dataframe[value_col] = pd.to_numeric(dataframe[value_col], 
                                             errors='coerce')
        dataframe[total_col] = pd.to_numeric(dataframe[total_col], 
                                             errors='coerce')
        
        result_df[percentage_col_name] = (
            dataframe[value_col] / dataframe[total_col]) * 100

    return result_df

# Selecting the right dominator
total_cols = ['All households', # For Households composition 
              'All households',
              'All people.3', # For Health
              'All people.3',
              'All people aged 16 to 74', # For Economic activity
              'All people aged 16 to 74',
              'All occupied household spaces', # For Central heating 
              'All occupied household spaces' ]

# Selecting the variables that we are interested in
# I also included contradictory variables 
# to the variables in the DEA analysis
# It could capture a more complete and 
# accurate picture of neighborhood vulnerability
value_cols = [
'One family only: Married or same-sex civil partnership couple: With dependent children',
              'One person household: Aged 65 and over',
              'Bad health',
              'Good health',
              'Economically active: Unemployed',
              'Economically active: Employee: Full-time',
              'No central heating',
              'Gas central heating']


result_dataframe = calculate_percentages(merged_data, 
                                         total_cols, 
                                         value_cols)
result_dataframe.head()

In [ ]:
# To Concatenate date
concatenated_df = pd.concat([merged_data, 
                             result_dataframe], 
                            axis=1, 
                            ignore_index=False)
concatenated_df.head()

In [ ]:
# Subsetting the attributes to make it clear
keep_cols= [
    'code',
    'Popcount',
    'HHcount',
    'DataZone',
    'geometry',
    'One family only: Married or same-sex civil partnership couple: With dependent children',
    'One person household: Aged 65 and over',
    'Bad health',
    'Good health',
    'Economically active: Unemployed',
    'Economically active: Employee: Full-time',
    'No central heating',
    'Gas central heating'
]

EDI_SUB_data = concatenated_df[keep_cols]
EDI_SUB_data.head()

In [ ]:
short_column_names = {

'One family only: Married or same-sex civil partnership couple: With dependent children': 
    'ODC',
    'One person household: Aged 65 and over': '65up',
    'Bad health': 'B_H',
    'Good health': 'G_H',
    'Economically active: Unemployed': 'EA_U',
    'Economically active: Employee: Full-time': 'EA_EF',
    'No central heating': 'N_H',
    'Gas central heating': 'Gs_H'
}

EDI_SUB_data = EDI_SUB_data.rename(columns=short_column_names)

In [ ]:
# Checking the data type
EDI_SUB_data.info()

In [ ]:
# Some of the columns turn out to be int64 
# Now we have to switch it to float64
import warnings
import pandas as pd
import numpy as np

# to ignore the warning for creating a copy
warnings.filterwarnings("ignore", 
                        category=pd.errors.SettingWithCopyWarning)

cols_to_convert = ['G_H', 'N_H', 'Gs_H','65up']

EDI_SUB_data[cols_to_convert] = EDI_SUB_data[
cols_to_convert].replace('-',np.nan).astype('float64')

print(EDI_SUB_data.dtypes)

In [ ]:
# Includes all numeric columns
numeric_columns = EDI_SUB_data.select_dtypes(include='float64')

# Convrt data into z score
z_score_df = (numeric_columns - numeric_columns.mean()) / numeric_columns.std(ddof=0)
z_score_df.head()

In [ ]:
# Explore the correlation between attributes
corr = z_score_df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
import matplotlib.pyplot as plt

plt.matshow(z_score_df.corr())
plt.show()

In [ ]:
# Checking for multicollinearity between variables
# Once higher than the threshold, it counts as multicollinearity

threshold = 0.7
highly_correlated = (corr.abs() > threshold) & (corr.abs() < 1.0)


plt.figure(figsize=(10, 8))
sns.heatmap(highly_correlated, cmap='coolwarm', cbar=False, annot=True)
plt.title('Highly Correlated Variables')
plt.show()

No multicollinearity is observesd.

In [ ]:
#Checking for any NaN values
contains_nan = z_score_df.isna().any().any()

if contains_nan:
    print("Contains NaN values....of course it does")
else:
    print("No NaN values.That couldn't be true?!")

In [ ]:
# Replace NaN value with mean
z_score_df.fillna(z_score_df.mean(), inplace=True)

In [ ]:
z_score_df.head()

In [ ]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist, pdist

# 10 clusters defined subjectivly 
kmeans = KMeans(n_clusters=10)
kmeans.fit(z_score_df)
labels = kmeans.predict(z_score_df)
cluster_centres = kmeans.cluster_centers_
z_score_df['Cluster'] = kmeans.labels_

In [ ]:
z_score_df.head()

In [ ]:
plt.hist(labels)

In [ ]:
# Running the Elbow Method for Optimal k
Sum_of_squared_distances = []

K_range = range(1,15)

for k in K_range:
 km = KMeans(n_clusters=k)
 km = km.fit(z_score_df)
 Sum_of_squared_distances.append(km.inertia_)
    
plt.plot(K_range, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
import numpy as np
def elbow(dataframe, n):
    kMeansVar = [KMeans(n_clusters=k).fit(dataframe.values) for k in range(1, n)] 
    centroids = [X.cluster_centers_ for X in kMeansVar]
    k_euclid = [cdist(dataframe.values, cent) for cent in centroids]
    dist = [np.min(ke, axis=1) for ke in k_euclid]
    wcss = [sum(d**2) for d in dist]
    tss = sum(pdist(dataframe.values)**2)/dataframe.values.shape[0]
    bss = tss - wcss
    plt.plot(bss)
    plt.show()
 
elbow(z_score_df,15)

In [ ]:
# Setting the Optimal k as 6 by the Elbow method.

kmeans = KMeans(n_clusters=6)
kmeans.fit(z_score_df)
labels = kmeans.predict(z_score_df)
cluster_centres = kmeans.cluster_centers_

z_score_df['Cluster'] = kmeans.labels_

In [ ]:
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

plt.figure(figsize=(8,12))

kmeans = KMeans(n_clusters=6)
clusters = kmeans.fit_predict(z_score_df)

z_score_df['Cluster'] = clusters

scaler = StandardScaler()
stand_data_scaled = scaler.fit_transform(z_score_df)

# Running the PCA and plotting a scatter plot
pca = PCA(n_components=2).fit(stand_data_scaled)
pca_result = pca.transform(stand_data_scaled)

#Percentage of variance explained by each of the selected components.
variance_ratio = pca.explained_variance_ratio_


fig = px.scatter(x=pca_result[:, 0], y=pca_result[:, 1], color=clusters,
                 labels={'color': 'Cluster'},
                 opacity=0.7,
                 width=800, 
                 height=800)

plt.tight_layout()
fig.show()

print(f"These two components explain {(variance_ratio.sum()*100):.2f}% of the point variability.")


In [ ]:
# examining the variability provided by each component.

kmeans = KMeans(n_clusters=6)
clusters = kmeans.fit_predict(z_score_df)

z_score_df['Cluster'] = clusters

# Standardize the data before conducting PCA
scaler = StandardScaler()
stand_data_scaled = scaler.fit_transform(z_score_df)

# PCA
pca = PCA(n_components=2).fit(stand_data_scaled)
pca_result = pca.transform(stand_data_scaled)

#Percentage of variance explained by each of the selected components.
variance_ratio = pca.explained_variance_ratio_

plt.figure(figsize=(10, 6))
sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1], hue=clusters, palette='viridis', s=50, alpha=0.7)
plt.title('Cluster Plot against 1st 2 Principal Components')
plt.xlabel(f'Principal Component 1 variation: {variance_ratio[0]*100:.2f}%')
plt.ylabel(f'Principal Component 2 variation: {variance_ratio[1]*100:.2f}%')
plt.legend(title='Clusters')
plt.show()


In [ ]:
# KMeans clustering
kmeans = KMeans(n_clusters=6)
clusters = kmeans.fit_predict(z_score_df)

# Get the cluster centers
cluster_centers = kmeans.cluster_centers_
cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=z_score_df.columns)

cluster_centers.head(6)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

num_features = len(cluster_centers.columns)
theta = np.linspace(0, 2 * np.pi, num_features, endpoint=True)

fig, axes = plt.subplots(2, 3, subplot_kw={'projection': 'polar'}, figsize=(18, 10))
axes = axes.flatten()

for i in range(6):
    ax = axes[i]
    center_values = cluster_centers.iloc[i, :].values

    ax.plot(theta, center_values, linewidth=2, marker='o', label=f'Cluster {i}')
    ax.fill(theta, center_values, alpha=0.25)
    ax.plot(theta, np.zeros_like(center_values), color='red', linestyle='--', label='Average')

    ax.set_xticks(theta)
    ax.set_xticklabels(cluster_centers.columns, fontsize=9)

    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True)


plt.suptitle("")  
plt.tight_layout()
plt.show()


Cluster 0: Unemployed Household in poor health condition
Cluster 1: Best condition families with children dependent (with good health, central heating and a full-time job)
Cluster 2: Average families with children dependent
Cluster 3: Worst condition Household (both live-alone elderly and families with children) in poor health condition, without central heating and unemployed.
Cluster 4: Families with children working full-time yet living without heating
Cluster 5: Best condition Household (both live-alone elderly and families with children), with good health, central heating and a full-time job.



In [ ]:
list(z_score_df.columns)

In [ ]:
z_score_df.drop(['ODC', 
                 '65up', 
                 'B_H', 
                 'G_H', 
                 'EA_U', 
                 'EA_EF', 
                 'N_H', 
                 'Gs_H'], axis=1, inplace=True)
z_score_df.info()

In [ ]:
final_df = pd.concat([EDI_SUB_data, z_score_df], axis=1, ignore_index=False)
final_df.head()

In [ ]:
final_df['Cluster'] = final_df['Cluster'].astype('category')

# Use explore with categorical=True.
final_df.explore(
    column='Cluster',
    cmap='Set1',
    tiles='CartoDB positron',
    categorical=True,
    legend_kwds={'title': 'Cluster', 'labels': ['0', '1', '2', '3', '4', '5']}
)


In [ ]:
final_df['Cluster'] = final_df['Cluster'].astype(str)


def rename_column(x): 
    x = x.replace("0", "Unwell & Unemployed")
    x = x.replace("1", "Comfortable Families")
    x = x.replace("2", "Ordinary Working Families")
    x = x.replace("3", "Vulnerable Households")
    x = x.replace("4", "Working but Energy-Poor")
    x = x.replace("5", "Healthy & Secure Households")
    # Add more values as you need. As many as clusters you have
    return x

final_df['Cluster'] = final_df['Cluster'].apply(rename_column)

In [ ]:
final_df['Cluster'] = final_df['Cluster'].astype('category')


final_df.explore(
    column='Cluster',
    cmap='Set1',
    tiles='CartoDB positron',
    categorical=True,
    legend_kwds={'title': 'Cluster', 'labels': ['0', '1', '2', '3', '4', '5']}
)


In service planning, this Geodemographic Classification helps identify the character of the target service user and indicate types of service that should be enhanced. For Cluster 0, the health condition and the unemployment situation would have to be addressed. Services such as health care allowance or employment programs have to be prioritized in consideration. Whereas, Cluster 1 is families with dependent children. Although they are rather well-off, services such as day care and education could still be improved for future development. The situation of Cluster 1 is quite similar to Cluster 3. As for Cluster 4, support in fostering living conditions could be considered. Last but not least, Cluster 5 is facing population aging; the current condition is rather secure, and service could focus on community connecting or building a nursing home.  Nonetheless, Cluster is not a clear-cut. The algorithm put each household into a single category. Yet in reality, households exist along a spectrum of socio-economic and environmental conditions. Furthermore, the choice of variables, number of clusters, and even the clustering method itself influence how these groupings emerge. Therefore, the application of cluster should be under careful consideration.